In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select

import time
from datetime import datetime

import requests
from bs4 import BeautifulSoup
import re as re
import time
import pandas as pd
import numpy as np

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords

from wordcloud import WordCloud
from langdetect import detect
from textblob import TextBlob

In [2]:
def url_request(url):
    driver = webdriver.Chrome()
    driver.get(url)
   
    return driver

In [3]:
driver= url_request('https://www.bbc.com/news')

In [4]:
def keyword(word):
    time.sleep(5)
    cookies_accept = driver.find_element(By.CLASS_NAME, 'fc-button').click()
    cookies_2= driver.find_element(By.CLASS_NAME, 'banner-button').click()
    
    
    time.sleep(3)
    search_bar  = driver.find_element(By.LINK_TEXT, 'Search BBC').click()
    search_bar  = driver.find_element(By.ID, 'search-input')
    search_bar.send_keys(word)
    search_bar.send_keys(Keys.RETURN)
    
    link_lst=[]
    

    while len(link_lst)<10:
        lnks           = driver.find_elements(By.TAG_NAME,"a")
        lst_bbc        = [lnk.get_attribute('href') for lnk in lnks]
        lst_bbc_       = [url for url in lst_bbc if '/news/' in url and '/help' not in url and '/live/' not in url]
        
        try:
            search_bar = driver.find_element(By.LINK_TEXT, 'next page').click()
        except:
            break

        for i in lst_bbc_:
            link_lst.append(i)

        link_lst= list(set(link_lst))

        list_title=[]
        list_soup=[]
        for url in link_lst:
            html = requests.get(url)
            soup = BeautifulSoup(html.content, "html.parser")
            article=soup.getText().replace('\'', "´").strip()
            article = article[:article.find('More on this story')]
            title = soup.title.string
            list_soup.append(article)
            list_title.append(title)

        day_list=[]
        month_list=[]
        year_list=[]
        for url in link_lst:
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            time_tag = soup.find('time')
            timestamp = time_tag['datetime']
            dt = datetime.fromisoformat(timestamp[:-1])
            date_str = dt.strftime("%d-%m-%Y")


            day, month, year = date_str.split('-')

            day_list.append(day)
            month_list.append(month)
            year_list.append(year)




        dict_bbc={'title': list_title,
                'article': list_soup,
                 'link':link_lst,
                 'day':day_list,
                 'month': month_list,
                 'year': year_list}
        df=pd.DataFrame(dict_bbc)
        driver.close()
        return df


In [5]:
df_bbc=keyword('Russia')

In [18]:
df_bbc.to_csv('bbc_russia.csv',index=False)  

In [19]:
df_bbc=pd.read_csv('bbc_russia.csv')

In [21]:
df_bbc

,title,article,link,day,month,year
0,Ukraine war: Russia's Wagner boss suggests 'be...,Ukraine war: Russia´s Wagner boss suggests ´be...,https://www.bbc.co.uk/news/world-europe-64859780,6,3,2023
1,Ukraine in maps: Tracking the war with Russia ...,Ukraine in maps: Tracking the war with Russia ...,https://www.bbc.co.uk/news/world-europe-60506682,9,3,2023
2,Russia country profile - BBC News,Russia country profile - BBC NewsBBC HomepageS...,https://www.bbc.co.uk/news/world-europe-17839672,6,3,2023
3,Ukraine war: Russia fires hypersonic missiles ...,Ukraine war: Russia fires hypersonic missiles ...,https://www.bbc.co.uk/news/world-europe-64903202,9,3,2023
4,Ukraine war: Why Russia’s infrastructure strik...,Ukraine war: Why Russia’s infrastructure strik...,https://www.bbc.co.uk/news/world-europe-64900098,9,3,2023
5,Russia media guide - BBC News,Russia media guide - BBC NewsBBC HomepageSkip ...,https://www.bbc.co.uk/news/world-europe-17840134,6,3,2023
6,Ukraine war: Why Bakhmut matters for Russia an...,Ukraine war: Why Bakhmut matters for Russia an...,https://www.bbc.co.uk/news/world-europe-64877991,9,3,2023
